# Kafka brokerの構築手順

In [1]:
set -o pipefail

## Ansibleの設定

ansibleをつかってKafka brokerクラスタ(とzookeeperクラスタ)を構築する。

ansibleのインベントリファイルを作成する。

In [2]:
cat >inventory.yml <<EOF
all:
    children:
        kafka:
            hosts:
                server1.example.jp:
                #ここにKafkaをうごかすホストを羅列する。行末のコロンを忘れずに
            vars:
                ansible_user: piyo  #実行ユーザは変更する
                ansible_ssh_private_key_file: ~/.ssh/id_rsa
                ansible_python_interpreter: /usr/bin/python3
        zookeeper:
            hosts:
                server1.example.jp:
                #ここにZookeeperをうごかすホストを羅列する。行末のコロンを忘れずに
            vars:
                ansible_user: piyo  #実行ユーザは変更する
                ansible_ssh_private_key_file: ~/.ssh/id_rsa
                ansible_python_interpreter: /usr/bin/python3
EOF

ansibleの設定ファイルを作成する。

In [3]:
cat >ansible.cfg <<EOF
[defaults]
command_warnings = False
inventory = ./inventory.yml
EOF

ansibleを通じてzookeeperとkafkaを実行するホストにアクセスできるのを確認する。

In [4]:
ansible all -m ping

server1.example.jp | SUCCESS => {
    "changed": false,
    "ping": "pong"
}


Dockerがインストールされているのを確認する。

In [5]:
ansible all -m command -a "docker version"

server1.example.jp | CHANGED | rc=0 >>
Client: Docker Engine - Community
 Version:           19.03.5
 API version:       1.40
 Go version:        go1.12.12
 Git commit:        633a0ea
 Built:             Wed Nov 13 07:25:41 2019
 OS/Arch:           linux/amd64
 Experimental:      false

Server: Docker Engine - Community
 Engine:
  Version:          19.03.5
  API version:      1.40 (minimum version 1.12)
  Go version:       go1.12.12
  Git commit:       633a0ea
  Built:            Wed Nov 13 07:24:18 2019
  OS/Arch:          linux/amd64
  Experimental:     false
 containerd:
  Version:          1.2.10
  GitCommit:        b34a5c8af56e510852c35414db4c1f4fa6172339
 runc:
  Version:          1.0.0-rc8+dev
  GitCommit:        3e425f80a8c931f88e6d94a8c831b9d5aa481657
 docker-init:
  Version:          0.18.0
  GitCommit:        fec3683


## ZooKeeperクラスタの構築

zookeeperのdockerイメージ名とポート番号を設定する。
PPORT,LPORT,CPORTはほかのサービスのポート番号とぶつかっているのでなければ変更する必要はない。
zookeeperの仕様によりCPORTは変更できない。

In [6]:
DOCKER_IMAGE="zookeeper"

ZK_PPORT=12888      # peer
ZK_LPORT=13888      # leader
ZK_CPORT=2181      # client

zookeeperを起動するスクリプトを生成する。

In [7]:
LIST_ZOOKEEPER_HOSTS="$(ansible-inventory --list  | jq  -r '.zookeeper.hosts|.[]')"
list_zookeeper_hosts() {
    echo "$LIST_ZOOKEEPER_HOSTS"
}

print_servers() {
    local MYID="$1"
    local HOST
    local ID=1
    local SERVER
    list_zookeeper_hosts | while read HOST; do
        if [ "$ID" = "$MYID" ]; then
            local ANYADDR="0.0.0.0"
            HOST="$ANYADDR"
        fi
        printf "server.$ID=$HOST:$ZK_PPORT:$ZK_LPORT "
        ID=$((ID + 1))
    done
    printf "\n"
}

print_docker_run() {
    local DIR="$1"
    local ID=1
    list_zookeeper_hosts | while read HOST; do
        #local NAME="sinetstream-zookeeper-$ID"
        local NAME="sinetstream-zookeeper"
        local SERVERS="$(print_servers "$ID")"
        {
            printf "docker run --rm --detach --name '$NAME' --env 'ZOO_MY_ID=$ID' --env 'ZOO_SERVERS=$SERVERS' --publish $ZK_PPORT:$ZK_PPORT --publish $ZK_LPORT:$ZK_LPORT --publish $ZK_CPORT:$ZK_CPORT $DOCKER_IMAGE"
        } > "$DIR/zookeeper-docker_run-${HOST}.sh"
        ID=$((ID + 1))
    done
}

mkdir -p tmp  &&
rm -f tmp/*.sh  &&
print_docker_run tmp  &&
ls -l tmp/*.sh

-rw-r--r-- 1 jovyan users 199 May 12 18:40 tmp/zookeeper-docker_run-server1.example.jp.sh


ansibleをつかってzookeeperサーバーを起動する。

In [8]:
ansible zookeeper -m script -a 'tmp/zookeeper-docker_run-{{inventory_hostname}}.sh'

server1.example.jp | CHANGED => {
    "changed": true,
    "rc": 0,
    "stderr": "Shared connection to server1.example.jp closed.\r\n",
    "stderr_lines": [
        "Shared connection to server1.example.jp closed."
    ],
    "stdout": "5b920416deb6d59169840a4643fc7ed1e6a0311fd5824f532b19aafbe03858cf\r\n",
    "stdout_lines": [
        "5b920416deb6d59169840a4643fc7ed1e6a0311fd5824f532b19aafbe03858cf"
    ]
}


In [9]:
ansible zookeeper -m command -a 'docker ps --filter "name=sinetstream-zookeeper"'

server1.example.jp | CHANGED | rc=0 >>
CONTAINER ID        IMAGE               COMMAND                  CREATED             STATUS              PORTS                                                                                            NAMES
5b920416deb6        zookeeper           "/docker-entrypoint.…"   3 seconds ago       Up 1 second         2888/tcp, 0.0.0.0:2181->2181/tcp, 0.0.0.0:12888->12888/tcp, 0.0.0.0:13888->13888/tcp, 3888/tcp   sinetstream-zookeeper


## Kafkaクラスタ

公式のKafka一式をダウンロードする。
手元でダウンロードしてから各ホストにコピーする。

In [10]:
KAFKA="kafka_2.12-2.4.1"

In [11]:
wget --mirror http://ftp.kddilabs.jp/infosystems/apache/kafka/2.4.1/$KAFKA.tgz

--2020-05-12 18:40:18--  http://ftp.kddilabs.jp/infosystems/apache/kafka/2.4.1/kafka_2.12-2.4.1.tgz
Resolving ftp.kddilabs.jp (ftp.kddilabs.jp)... 192.26.91.193, 2001:200:601:10:206:5bff:fef0:466c
Connecting to ftp.kddilabs.jp (ftp.kddilabs.jp)|192.26.91.193|:80... connected.
HTTP request sent, awaiting response... 304 Not Modified
File ‘ftp.kddilabs.jp/infosystems/apache/kafka/2.4.1/kafka_2.12-2.4.1.tgz’ not modified on server. Omitting download.



In [12]:
ansible kafka -m command -a "mkdir -p \$PWD/sinetstream-kafka"

server1.example.jp | CHANGED | rc=0 >>



In [13]:
ansible kafka -m copy -a "src=$KAFKA.tgz dest=\$PWD/sinetstream-kafka/"

server1.example.jp | SUCCESS => {
    "changed": false,
    "checksum": "d043d80b62dff190c22d11f4afbe8c59827ba7a5",
    "dest": "/home/piyo/sinetstream-kafka/kafka_2.12-2.4.1.tgz",
    "gid": 1004,
    "group": "piyo",
    "mode": "0664",
    "owner": "piyo",
    "path": "/home/piyo/sinetstream-kafka/kafka_2.12-2.4.1.tgz",
    "size": 62358954,
    "state": "file",
    "uid": 1004
}


### KafkaブローカーをうごかすCentOSのコンテナを作成

認証方法をどれかひとつ選択する。

In [14]:
#KAFKA_AUTH=SSL        # SSL/TLS認証（クライアント認証）; 通信にTLSをつかい、認証に証明書をつかう
KAFKA_AUTH=SASL_SSL_SCRAM  # SCRAM認証/TLS; 通信にTLSをつかい、認証にSCRAM(パスワード)をつかう
#KAFKA_AUTH=SASL_SSL_PLAIN  # パスワード認証/TLS; 通信にTLSをつかい、認証に平文パスワードをつかう
#KAFKA_AUTH=PLAINTEXT # 通信は暗号化されず、認証もない ※つかってはいけない

truststore/keystoreを保護するためのパスワードを設定する。パスワードは適当に強度の高い文字列を指定する。

In [15]:
TRUSTSTORE_PASSWORD="trust-pass-00"
KEYSTORE_PASSWORD="key-pass-00"

SCAM認証やパスワード認証を使う場合には、ユーザーのリストと各ユーザのパスワードを設定する。
SSL/TLS認証を使う場合はパスワードは設定しなくてよく、ユーザのリストだけを設定する。SSL/TLS認証でのユーザ名は証明書のCommon Nameである。

ユーザ `admin` はkafkaブローカ間の通信につかう特別なユーザなので消してはいけない。
パスワードは十分な強度をもったものに変更すべきである。

In [16]:
USER_LIST="user01 user02 user03 CN=client0,C=JP"
PASSWORD_admin="admin-pass"
PASSWORD_user01="user01-pass"
PASSWORD_user02="user02-pass"
PASSWORD_user03="user03-pass"

認可(ACL)の設定。

In [17]:
KAFKA_ACL_DEFAULT_TO_ALLOW="false"  # trueに設定するとACLが設定されていないユーザはアクセスが許可される。
ACL_user01="readwrite"
ACL_user02="write"
ACL_user03="read"
ACL_CN_client0_C_JP="readwrite"  # 英数字以外は _ に置き換えて

In [18]:
KAFKA_PORT_SSL=9093    
KAFKA_PORT_SASL_SSL=9093

認証方法の詳細なパラメータを設定する。

In [19]:
SCRAM_MECHANISM="SCRAM-SHA-256"

Kafkaブローカを動かすコンテナを作る。

In [20]:
ansible kafka -m command -a "docker run \
    --detach \
    --interactive \
    --net host \
    --name sinetstream-kafka \
    --volume \$PWD/sinetstream-kafka:/sinetstream-kafka \
    centos:7"

server1.example.jp | CHANGED | rc=0 >>
642b24558f7629a0a071f65bac082644fd61fc40dac92d912b08a75a097f7268


In [21]:
ansible kafka -m command -a "docker exec sinetstream-kafka true"

server1.example.jp | CHANGED | rc=0 >>



コンテナにKafkaの実行に必要なソフトウェアをインストールする。

In [22]:
ansible kafka -m command -a "docker exec sinetstream-kafka yum update -y"

server1.example.jp | CHANGED | rc=0 >>
Loaded plugins: fastestmirror, ovl
Determining fastest mirrors
 * base: ty1.mirror.newmediaexpress.com
 * extras: ty1.mirror.newmediaexpress.com
 * updates: ty1.mirror.newmediaexpress.com
Resolving Dependencies
--> Running transaction check
---> Package acl.x86_64 0:2.2.51-14.el7 will be updated
---> Package acl.x86_64 0:2.2.51-15.el7 will be an update
---> Package bash.x86_64 0:4.2.46-33.el7 will be updated
---> Package bash.x86_64 0:4.2.46-34.el7 will be an update
---> Package bind-license.noarch 32:9.11.4-9.P2.el7 will be updated
---> Package bind-license.noarch 32:9.11.4-16.P2.el7_8.2 will be an update
---> Package binutils.x86_64 0:2.27-41.base.el7 will be updated
---> Package binutils.x86_64 0:2.27-43.base.el7 will be an update
---> Package ca-certificates.noarch 0:2018.2.22-70.0.el7_5 will be updated
---> Package ca-certificates.noarch 0:2019.2.32-76.el7_7 will be an update
---> Package centos-release.x86_64 0:7-7.1908.0.el7.centos will be 

In [23]:
ansible kafka -m command -a "docker exec sinetstream-kafka yum install -y java-1.8.0-openjdk openssl"

server1.example.jp | CHANGED | rc=0 >>
Loaded plugins: fastestmirror, ovl
Loading mirror speeds from cached hostfile
 * base: ty1.mirror.newmediaexpress.com
 * extras: ty1.mirror.newmediaexpress.com
 * updates: ty1.mirror.newmediaexpress.com
Resolving Dependencies
--> Running transaction check
---> Package java-1.8.0-openjdk.x86_64 1:1.8.0.252.b09-2.el7_8 will be installed
--> Processing Dependency: java-1.8.0-openjdk-headless(x86-64) = 1:1.8.0.252.b09-2.el7_8 for package: 1:java-1.8.0-openjdk-1.8.0.252.b09-2.el7_8.x86_64
--> Processing Dependency: xorg-x11-fonts-Type1 for package: 1:java-1.8.0-openjdk-1.8.0.252.b09-2.el7_8.x86_64
--> Processing Dependency: libpng15.so.15(PNG15_0)(64bit) for package: 1:java-1.8.0-openjdk-1.8.0.252.b09-2.el7_8.x86_64
--> Processing Dependency: libjvm.so(SUNWprivate_1.1)(64bit) for package: 1:java-1.8.0-openjdk-1.8.0.252.b09-2.el7_8.x86_64
--> Processing Dependency: libjpeg.so.62(LIBJPEG_6.2)(64bit) for package: 1:java-1.8.0-openjdk-1.8.0.252.b09-2.el7_8

In [24]:
ansible kafka -m command -a "docker exec sinetstream-kafka tar xf /sinetstream-kafka/$KAFKA.tgz" &&
ansible kafka -m command -a "docker exec sinetstream-kafka ln -s /$KAFKA /kafka"

server1.example.jp | CHANGED | rc=0 >>

server1.example.jp | CHANGED | rc=0 >>



### Kafkaブローカの設定

kafkaブローカの設定ファイルを生成する。

In [25]:
LIST_KAFKA_HOSTS="$(ansible-inventory --list  | jq  -r '.kafka.hosts|.[]')"
list_kafka_hosts() {
    echo "$LIST_KAFKA_HOSTS"
}

print_server_properties() {
    local HOST="$1"
    local ID="$2"
    
    echo "broker.id=${ID}"
    
    local ZKHOST
    printf "zookeeper.connect="
    list_zookeeper_hosts | sed "s/\$/:${ZK_CPORT}/" | paste -s -d,

    printf "listeners="
    {
        case "$KAFKA_AUTH" in
        PLAINTEXT) echo "PLAINTEXT://:${KAFKA_PORT_PLAINTEXT}" ;;
        SSL)       echo "SSL://:${KAFKA_PORT_SSL}" ;;
        SASL_SSL*) echo "SASL_SSL://:${KAFKA_PORT_SASL_SSL}"
                   echo "SSL://:$((KAFKA_PORT_SASL_SSL+1))" ;;
        esac
    } | paste -s -d,
    
    printf "advertised.listeners="
    {
        case "$KAFKA_AUTH" in
        PLAINTEXT) echo "PLAINTEXT://${HOST}:${KAFKA_PORT_PLAINTEXT}" ;;
        SSL)       echo "SSL://${HOST}:${KAFKA_PORT_SSL}" ;;
        SASL_SSL*) echo "SASL_SSL://${HOST}:${KAFKA_PORT_SASL_SSL}"
                   echo "SSL://${HOST}:$((KAFKA_PORT_SASL_SSL+1))" ;; # for inter-broker
        esac
    } | paste -s -d,
    

    # CA証明書の設定
    echo "ssl.truststore.location=/sinetstream-kafka/truststore.p12"
    echo "ssl.truststore.password=${TRUSTSTORE_PASSWORD}"
    echo "ssl.truststore.type=pkcs12"
    # サーバー秘密鍵の設定
    echo "ssl.keystore.location=/sinetstream-kafka/keystore.p12"
    echo "ssl.keystore.password=${KEYSTORE_PASSWORD}"
    echo "ssl.keystore.type=pkcs12"
        
    case "$KAFKA_AUTH" in
    SSL)
        # SSL/TLS認証（クライアント認証）
        echo "ssl.client.auth=required"
        echo "security.inter.broker.protocol=SSL"
        ;;
    SASL_SSL_SCRAM)
        # SCRAM認証/TLS
        echo "ssl.client.auth=required"
        echo "security.inter.broker.protocol=SSL"
        echo "sasl.enabled.mechanisms=${SCRAM_MECHANISM}"
        #echo "sasl.mechanism.inter.broker.protocol=${SCRAM_MECHANISM}"
        local scram_mechanism="$(echo "${SCRAM_MECHANISM}" | tr '[A-Z]' '[a-z]')"
        echo "listener.name.sasl_ssl.${scram_mechanism}.sasl.jaas.config=org.apache.kafka.common.security.scram.ScramLoginModule required \\"
        echo "    username=admin password=${PASSWORD_admin};"
        ;;
    SASL_SSL_PLAIN)
        # パスワード認証/TLS
        echo "ssl.client.auth=required"
        echo "security.inter.broker.protocol=SSL"
        echo "sasl.enabled.mechanisms=PLAIN"
        #echo "sasl.mechanism.inter.broker.protocol=PLAIN"
        echo "listener.name.sasl_ssl.plain.sasl.jaas.config=org.apache.kafka.common.security.plain.PlainLoginModule required \\"
        echo "    username=admin password=${PASSWORD_admin} \\"
        local USER PASSWORD
        for USER in ${USER_LIST}; do
            eval PASSWORD=\$PASSWORD_${USER}
            echo "    user_${USER}=\"${PASSWORD}\" \\"
        done
        echo "    ;"
        ;;
    esac
    
    # 認可
    echo "authorizer.class.name=kafka.security.auth.SimpleAclAuthorizer"  # ZooKeeperに記録されているACL設定による認可
    echo "allow.everyone.if.no.acl.found=${KAFKA_ACL_DEFAULT_TO_ALLOW}"
    echo "super.users=User:admin"  # adminには特権を与える
}

ID=1
tar x -f $KAFKA.tgz --to-stdout $KAFKA/config/server.properties >server.properties  &&
mkdir -p tmp  &&
rm -f tmp/*.properties  &&
list_kafka_hosts | while read HOST; do
    {
        cat server.properties
        print_server_properties "$HOST" "$ID"
    } >"tmp/server-${HOST}.properties"
    ID=$((ID + 1))
done

In [26]:
ls -l tmp/server-*.properties

-rw-r--r-- 1 jovyan users 7644 May 12 18:44 tmp/server-server1.example.jp.properties


kafkaブローカの設定ファイルを各ホストにコピーする。

In [27]:
ansible kafka -m copy -a "src=tmp/server-{{inventory_hostname}}.properties dest=\$PWD/sinetstream-kafka/server.properties"

server1.example.jp | CHANGED => {
    "changed": true,
    "checksum": "8db15549d6e82be4c5c3a93360f97de6cfbf0573",
    "dest": "/home/piyo/sinetstream-kafka/server.properties",
    "gid": 1004,
    "group": "piyo",
    "md5sum": "1f3d845ad03be8bfef632b72d54336ab",
    "mode": "0664",
    "owner": "piyo",
    "size": 7644,
    "src": "/home/piyo/.ansible/tmp/ansible-tmp-1589276689.879836-22666949452525/source",
    "state": "file",
    "uid": 1004
}


### SSL/TLSのための証明書を設定

opensslをつかってPEM形式の証明書をkafkaブローカが扱えるPKCS#12(p12)形式に変換する。

CA証明書・サーバ秘密鍵・サーバ証明書をkafkaブローカの動かすコンテナ内にコピーする。

自己署名CA証明書の場合はCA秘密鍵もコピーする。

In [28]:
CA_CERT_PATH=./cacert.pem
CA_KEY_PATH=NONE  
CA_KEY_PATH=./cakey.pem  # CA証明書が自己署名の場合はCA秘密鍵も指定する

BROKER_CERT_PATH=./broker.crt
BROKER_KEY_PATH=./broker.key

# 以下、変更しなくてよい
CA_CERT_FILE=$(basename "${CA_CERT_PATH}")
BROKER_CERT_FILE=$(basename "${BROKER_CERT_PATH}")
BROKER_KEY_FILE=$(basename "${BROKER_KEY_PATH}")
if [ "x$CA_KEY_PATH" != "xNONE" ]; then
    CA_KEY_FILE=$(basename "${CA_KEY_PATH}")
else
    CA_KEY_FILE=""
fi

In [29]:
ansible kafka -m copy -a "src=${CA_CERT_PATH} dest=\$PWD/sinetstream-kafka/${CA_CERT_FILE}" &&
ansible kafka -m copy -a "src=${BROKER_CERT_PATH} dest=\$PWD/sinetstream-kafka/${BROKER_CERT_FILE}" &&
ansible kafka -m copy -a "src=${BROKER_KEY_PATH} dest=\$PWD/sinetstream-kafka/${BROKER_KEY_FILE}" &&
if [ -n "${CA_KEY_FILE}" ]; then
    ansible kafka -m copy -a "src=${CA_KEY_PATH} dest=\$PWD/sinetstream-kafka/${CA_KEY_FILE}"
fi

server1.example.jp | SUCCESS => {
    "changed": false,
    "checksum": "43513e67aa1278fdd15ad23304971edc3f6dda52",
    "dest": "/home/piyo/sinetstream-kafka/cacert.pem",
    "gid": 1004,
    "group": "piyo",
    "mode": "0664",
    "owner": "piyo",
    "path": "/home/piyo/sinetstream-kafka/cacert.pem",
    "size": 4349,
    "state": "file",
    "uid": 1004
}
server1.example.jp | SUCCESS => {
    "changed": false,
    "checksum": "d92b90e240f0bf59677367354ffe2ce6e5f5c8c6",
    "dest": "/home/piyo/sinetstream-kafka/broker.crt",
    "gid": 1004,
    "group": "piyo",
    "mode": "0664",
    "owner": "piyo",
    "path": "/home/piyo/sinetstream-kafka/broker.crt",
    "size": 4389,
    "state": "file",
    "uid": 1004
}
server1.example.jp | SUCCESS => {
    "changed": false,
    "checksum": "cf6364f56c6ec29b2acdb40e3ede96fe77821585",
    "dest": "/home/piyo/sinetstream-kafka/broker.key",
    "gid": 1004,
    "group": "piyo",
    "mode": "0664",
    "owner": "piyo",
    "path": "/home/piyo/si

CA証明書を変換してtruststoreに登録する。

In [30]:
ansible kafka -m command -a "docker exec sinetstream-kafka \
  openssl pkcs12 -export \
    -in sinetstream-kafka/${CA_CERT_FILE} \
    ${CA_KEY_FILE:+-inkey sinetstream-kafka/${CA_KEY_FILE}} \
    -name private-ca \
    -CAfile sinetstream-kafka/${CA_CERT_FILE}\
    -caname private-ca \
    -out sinetstream-kafka/truststore.p12 \
    -passout pass:${TRUSTSTORE_PASSWORD}" &&
ansible kafka -m command -a "docker exec sinetstream-kafka \
  openssl pkcs12 -in sinetstream-kafka/truststore.p12 -passin pass:${TRUSTSTORE_PASSWORD} -info -noout"

server1.example.jp | CHANGED | rc=0 >>

server1.example.jp | CHANGED | rc=0 >>
MAC Iteration 2048
MAC verified OK
PKCS7 Encrypted data: pbeWithSHA1And40BitRC2-CBC, Iteration 2048
Certificate bag
PKCS7 Data
Shrouded Keybag: pbeWithSHA1And3-KeyTripleDES-CBC, Iteration 2048


サーバ秘密鍵・サーバ証明書・CA証明書を変換してkeystoreに登録する。

In [31]:
ansible kafka -m command -a "docker exec sinetstream-kafka \
  openssl pkcs12 -export \
    -in sinetstream-kafka/${BROKER_CERT_FILE} \
    -inkey sinetstream-kafka/${BROKER_KEY_FILE} \
    -name broker \
    -CAfile sinetstream-kafka/${CA_CERT_FILE} \
    -caname private-ca \
    -out sinetstream-kafka/keystore.p12 \
    -passout pass:${KEYSTORE_PASSWORD}" &&
ansible kafka -m command -a "docker exec sinetstream-kafka \
  openssl pkcs12 -in sinetstream-kafka/keystore.p12 -passin pass:${KEYSTORE_PASSWORD} -info -noout"

server1.example.jp | CHANGED | rc=0 >>

server1.example.jp | CHANGED | rc=0 >>
MAC Iteration 2048
MAC verified OK
PKCS7 Encrypted data: pbeWithSHA1And40BitRC2-CBC, Iteration 2048
Certificate bag
PKCS7 Data
Shrouded Keybag: pbeWithSHA1And3-KeyTripleDES-CBC, Iteration 2048


### SCRAM認証の設定

パスワードをzookeeperに保存する。

In [32]:
if [ "x$KAFKA_AUTH" = "xSASL_SSL_SCRAM" ]; then
    ZK1="$(list_zookeeper_hosts | head -1)"
    KAFKA1="$(list_kafka_hosts | head -1)"
    for USER in admin ${USER_LIST}; do
        eval PASSWORD=\$PASSWORD_${USER}
        ansible kafka --limit="${KAFKA1}" -m command -a "docker exec sinetstream-kafka \
            /kafka/bin/kafka-configs.sh --zookeeper ${ZK1}:${ZK_CPORT} --alter \
                --entity-type users \
                --entity-name ${USER} \
                --add-config 'SCRAM-SHA-256=[iterations=8192,password=${PASSWORD}]'"
    done &&
    ansible kafka -m command -a "docker exec sinetstream-kafka \
            /kafka/bin/kafka-configs.sh --zookeeper ${ZK1}:${ZK_CPORT} --describe --entity-type users"
fi

server1.example.jp | CHANGED | rc=0 >>
Completed Updating config for entity: user-principal 'admin'.
server1.example.jp | CHANGED | rc=0 >>
Completed Updating config for entity: user-principal 'user01'.
server1.example.jp | CHANGED | rc=0 >>
Completed Updating config for entity: user-principal 'user02'.
server1.example.jp | CHANGED | rc=0 >>
Completed Updating config for entity: user-principal 'user03'.
server1.example.jp | CHANGED | rc=0 >>
Completed Updating config for entity: user-principal 'CN=client0,C=JP'.
server1.example.jp | CHANGED | rc=0 >>
Configs for user-principal 'admin' are SCRAM-SHA-256=salt=bW5hOGN4MWhqdDFnb2x6M3JzZjZyNmkzdA==,stored_key=ABCz61QlROA189AQ08lwSJfccwrPHGfIsGbjJo0ytBQ=,server_key=VYAOL6tjsvIi/dBl9eLMFFo6eKRiuQPSYGbCaEFrj4w=,iterations=8192
Configs for user-principal 'user03' are SCRAM-SHA-256=salt=aWp1a2tmb3FsMzBxbjMxZ2lyOTV6dWltaw==,stored_key=GXmoPMFH43u6FrFAXGbE8vno8LutImQsdob86BptD/E=,server_key=/dtVne4q1dDvJgeDXryufJqaPCwxdyj8dnaDrWxKpME=,iterations=8

### 認可(ACL)の設定

ブローカがつかっているサーバ証明書のCommon Nameを設定する。ブローカ間通信の認可で必要となる。

In [33]:
ADMIN_USER="CN=server1.example.jp,C=JP"

In [34]:
ZK1="$(list_zookeeper_hosts | head -1)"
KAFKA1="$(list_kafka_hosts | head -1)"

ansible kafka --limit="${KAFKA1}" -m command -a "docker exec sinetstream-kafka \
    /kafka/bin/kafka-acls.sh --authorizer-properties zookeeper.connect=${ZK1}:${ZK_CPORT} \
    --add --allow-principal User:${ADMIN_USER} --cluster --operation All"  &&

for USER in ${USER_LIST}; do
    USER1=$(echo "$USER" | sed 's/[^[:alnum:]]/_/g')  # サニタイズ
    eval ACL=\$ACL_${USER1}
    case "${ACL}" in
    *write*)
        ansible kafka --limit="${KAFKA1}" -m command -a "docker exec sinetstream-kafka \
            /kafka/bin/kafka-acls.sh --authorizer-properties zookeeper.connect=${ZK1}:${ZK_CPORT} \
            --add --allow-principal User:${USER} \
            --producer --topic '*'"
            ;;
    esac
    case "${ACL}" in
    *read*)
        ansible kafka --limit="${KAFKA1}" -m command -a "docker exec sinetstream-kafka \
            /kafka/bin/kafka-acls.sh --authorizer-properties zookeeper.connect=${ZK1}:${ZK_CPORT} \
            --add --allow-principal User:${USER} \
            --consumer --topic '*' --group '*'"
        ;;
    esac
done 

server1.example.jp | CHANGED | rc=0 >>
Adding ACLs for resource `ResourcePattern(resourceType=CLUSTER, name=kafka-cluster, patternType=LITERAL)`: 
 	(principal=User:CN=server1.example.jp,C=JP, host=*, operation=ALL, permissionType=ALLOW) 

Current ACLs for resource `Cluster:LITERAL:kafka-cluster`: 
 	User:CN=server1.example.jp,C=JP has Allow permission for operations: All from hosts: * 
server1.example.jp | CHANGED | rc=0 >>
Adding ACLs for resource `ResourcePattern(resourceType=TOPIC, name=*, patternType=LITERAL)`: 
 	(principal=User:user01, host=*, operation=DESCRIBE, permissionType=ALLOW)
	(principal=User:user01, host=*, operation=WRITE, permissionType=ALLOW)
	(principal=User:user01, host=*, operation=CREATE, permissionType=ALLOW) 

Current ACLs for resource `Topic:LITERAL:*`: 
 	User:user01 has Allow permission for operations: Describe from hosts: *
	User:user01 has Allow permission for operations: Write from hosts: *
	User:user01 has Allow permission for operations: Create from ho

In [35]:
ansible kafka --limit="${KAFKA1}" -m command -a "docker exec sinetstream-kafka \
        /kafka/bin/kafka-acls.sh --authorizer-properties zookeeper.connect=${ZK1}:${ZK_CPORT} \
        --list"

server1.example.jp | CHANGED | rc=0 >>
Current ACLs for resource `Group:LITERAL:*`: 
 	User:user03 has Allow permission for operations: Read from hosts: *
	User:CN=client0,C=JP has Allow permission for operations: Read from hosts: *
	User:user01 has Allow permission for operations: Read from hosts: * 

Current ACLs for resource `Topic:LITERAL:*`: 
 	User:CN=client0,C=JP has Allow permission for operations: Read from hosts: *
	User:user01 has Allow permission for operations: Write from hosts: *
	User:user01 has Allow permission for operations: Create from hosts: *
	User:CN=client0,C=JP has Allow permission for operations: Describe from hosts: *
	User:CN=client0,C=JP has Allow permission for operations: Create from hosts: *
	User:CN=client0,C=JP has Allow permission for operations: Write from hosts: *
	User:user03 has Allow permission for operations: Read from hosts: *
	User:user02 has Allow permission for operations: Write from hosts: *
	User:user01 has Allow permission for operations: 

### Kafkaブローカー起動

In [36]:
ansible kafka -m command -a "docker exec --detach sinetstream-kafka \
   /kafka/bin/kafka-server-start.sh /sinetstream-kafka/server.properties"

server1.example.jp | CHANGED | rc=0 >>

